<a href="https://colab.research.google.com/github/ebknudsen/godiva_iv/blob/notebook/cad_to_openmc_godiva_iv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

## install the conda package manager

First we need to install the conda package manager. Inside a jupyter notebook this may be done as

In [ ]:
!pip install condacolab
import condacolab
condacolab.install()

In [ ]:
import condacolab
condacolab.check()

In a shell, one may grab the latest conda package manager from [miniconda](https://docs.anaconda.com/miniconda/) or one of its clones [mamba or micromamba](https://mamba.readthedocs.io/en/latest/). Follow the installation directions found. This yields a a package manager with environments allwing you to install a host of software.

Both `mamba` and `micromamba` are drop-in, open source replacements for conda but with improved package search facilities. `micromamba` is a lightweight implementation of mamba which works well for the simple case of installing software.  In the following we will use the command conda but this can always be substituted with mamba/micromamba.

## Install CAD_to_OpenMC
Now we will use the package manager to install CAD_to_OpenMC

In [ ]:
!conda install -y cad-to-openmc
!conda install openmc=0.15.0=dagmc_*

You should now have a functioning conda-environment from which you can run cad-to-openmc and OpenMC. This is done by importing the python module <code>CAD_to_OpenMC.assembly</code>.

## In a shell procedure
If outside a notebook setting you may do something similar (once conda has been installed):
`conda create -n my_environment python=3.11`\
`conda activate my_environment`\
`conda install -y cad-to-openmc`\
where the name "my_environment" is chosen arbitrarily. Any python version between 3.9 and 3.11 inclusive works. (We're working on 3.12 and 3.13)

In [ ]:
import CAD_to_OpenMC.assembly as ab

To get access to the Godiva_IV model we may simply clone a github repository where step-files containing both a simplifed version as well as a more detailed model of the Godiva IV device as described in the report "cite:Godiva_IV".

There are three step-files associated with each of the reactor models. Namely the reactor core, control rod, and burst rod. There are two control rods and a single burst rod, set in a 60 deg. symmetric pattern around the vertical axis through the reactor, center.

In the reactor experiments the control rods were used to set the overall working point of the reactor, and then the burst rod was used to reach criticality in short timespans, "bursts".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 #clone the godiva repo to get access to the step-files.
 !git clone https://www.github.com/ebknudsen/godiva_iv

In [ ]:
A=ab.Assembly(['/content/drive/sample_data/GIV_BR_detailed.step'])
B=ab.Assembly(['/content/drive/sample_data/GIV_CR_detailed.step'])
C=ab.assembly(['/content/drive/sample_data/GIV_core_detailed.step'])

In [ ]:
#now we run the meshing procedure on the models. At present the "regularly" best backend to use is the 'db'
A.run(backend='db', h5m_filename='GIV_BR_det.h5m')
B.run(backend='db', h5m_filename='GIV_CR_det.h5m')
C.run(backend='db', h5m_filename='GIV_core_det.h5m')

If all is well we should now have 3 files with the suffix ',h5m' in the working directory. These files contain al the vertices and triangles of a surface mesh-approximation of the reactor model.

If we examine the output from CAD_to_OpenMC we find that we have gotten reports from it about material-names, or tags, that it has attached to the various parts of the reactor,model. The material-names must coincide with the ones late inserted into the OpenMC-model.

Should you wish to do so, it is possible to "retag" parts, during the meshing process. This is done, either by supplying a python dictionary which will use regular expressions to match material tags, or a simple python list which will simply apply tags in the order they are encountered. This is useful, in the cases where no part name-tag is found in the .step-file. N.b. this procedure triggers a complete new mashing run, which can be computationally expensive (A future version of CAD_to_OpenMC will include a retagging procedure).

In [ ]:
#now we may start building our geometric model in openmc
#first build the geometry loading the surfaced meshed objects into separate DAGMCUniverse-objects which may be included inside a regular CSG-style geometry